### 確認單張照片

In [ ]:
from PIL import Image
import numpy as np
import torch
import autorootcwd
import matplotlib.pyplot as plt
from src.utils.image_encoder import ImageEncoder
from src.models.captcha_module import CaptchaModule  # 你自己的模型類
from src.data.components.build_dataset import DataParser


def predict_and_visualize(image_path, model_path):
    test_image, _ = DataParser().convert_image(image_path, image_path)
    test_image_tensor = (
        torch.tensor(test_image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    )  # 加入批次和通道維度

    model = CaptchaModule(None, None, None, None).load_from_checkpoint(model_path)
    model.eval()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    test_image_tensor = test_image_tensor.to(device)

    with torch.no_grad():
        output = model(test_image_tensor)
        pred_label = ImageEncoder().decode_output(output.squeeze())

    test_image = Image.open(image_path)
    plt.imshow(test_image, cmap="gray")
    plt.title(f"Predicted: {pred_label}")
    plt.axis("off")
    plt.show()


# 使用
image_path = "data/result_check/0JA8R.png"  # 你要預測的圖片路徑
model_path = "logs/captcha/runs/2023-08-27_10-56-04/tensorboard/version_0/checkpoints/last.ckpt"  # 你的模型路徑
predict_and_visualize(image_path, model_path)

### 確認準確率

In [ ]:
import math

from PIL import Image
import numpy as np
import torch
import autorootcwd
import matplotlib.pyplot as plt
from src.utils.image_encoder import ImageEncoder
from src.models.captcha_module import CaptchaModule
from src.data.components.build_dataset import DataParser


def predict_and_visualize(npz_path, ckpt_path):
    data = np.load(npz_path)
    test_images = data["images"]
    labels = data["labels"]

    model = CaptchaModule(None, None, None, None).load_from_checkpoint(ckpt_path)
    model.eval()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    num_images = len(test_images)
    correct_count = 0
    incorrect_images = []
    incorrect_labels = []
    incorrect_pred_labels = []

    for i in range(num_images):
        test_image = test_images[i]
        label = labels[i]
        test_image_tensor = (
            torch.tensor(test_image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        )  # 加入批次和通道維度
        test_image_tensor = test_image_tensor.to(device)

        with torch.no_grad():
            output = model(test_image_tensor)
            pred_label = ImageEncoder().decode_output(output.squeeze())

        if pred_label == label:
            correct_count += 1
        else:
            incorrect_images.append(test_image)
            incorrect_labels.append(label)
            incorrect_pred_labels.append(pred_label)

    accuracy = correct_count / num_images * 100
    num_incorrect = len(incorrect_images)
    num_cols = 2
    num_rows = math.ceil(num_incorrect / num_cols)

    _, axs = plt.subplots(num_rows, num_cols, figsize=(10, 5 * num_rows))
    for i in range(num_rows * num_cols):
        row = i // num_cols
        col = i % num_cols
        axs[row, col].axis("off")

        if i < num_incorrect:
            test_image = incorrect_images[i]
            label = incorrect_labels[i]
            pred_label = incorrect_pred_labels[i]

            axs[row, col].imshow(test_image)
            axs[row, col].set_title(f"Predicted: {pred_label}\nTrue: {label}")

    print(f"Accuracy: {accuracy:.2f}%\nCorrect: {correct_count}\nIncorrect: {num_incorrect}")
    # fig.suptitle(f'Accuracy: {accuracy:.2f}%', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


npz_path = "data/20230827_captcha/test/test.npz"
model_path = "logs/captcha_cnn/runs/2023-08-29_01-12-11"
ckpt_path = f"{model_path}/tensorboard/version_0/checkpoints/last.ckpt"
hparams_path = f"{model_path}/tensorboard/version_0/hparams.yaml"
device = "cpu"
predict_and_visualize(npz_path, ckpt_path)